In [ ]:
import re
import numpy as np
from collections import Counter
from pyvi import ViTokenizer
from sklearn.decomposition import PCA

In [ ]:
# Đọc và tiền xử lý dữ liệu
def preprocess_text(text):
    # Chuyển chữ thường
    text = text.lower()
    # Loại bỏ ký tự số
    text = re.sub(r'\d+', '', text)
    # Loại bỏ các ký tự đặc biệt và dấu câu
    text = re.sub(r'[^\w\s]', '', text)
    # Tách từ tiếng Việt
    tokenized_text = ViTokenizer.tokenize(text)
    return tokenized_text.split()

# Tạo từ điển từ vựng
def build_vocab(corpus):
    word_counts = Counter(corpus)
    # Sắp xếp từ vựng theo thứ tự chữ cái
    sorted_vocab = sorted(word_counts.items())
    vocab = {word: idx for idx, (word, _) in enumerate(sorted_vocab)}
    idx_to_word = {idx: word for word, idx in vocab.items()}
    return vocab, idx_to_word

# Tạo cặp từ trung tâm-ngữ cảnh
def generate_skipgram_pairs(corpus, vocab, window_size=1):
    pairs = []
    for i, word in enumerate(corpus):
        center_word_idx = vocab[word]
        for j in range(-window_size, window_size + 1):
            if j == 0 or i + j < 0 or i + j >= len(corpus):
                continue
            context_word_idx = vocab[corpus[i + j]]
            pairs.append((center_word_idx, context_word_idx))
    return pairs

# Đọc và tiền xử lý dữ liệu từ file
with open('./data/data.txt', 'r', encoding='utf-8') as f:
    text = f.read()

corpus = preprocess_text(text)
vocab, idx_to_word = build_vocab(corpus)
pairs = generate_skipgram_pairs(corpus, vocab)

In [ ]:
# Lưu từ vựng vào file
with open('./data/vocab.txt', 'w', encoding='utf-8') as f:
    for word, idx in vocab.items():
        f.write(f"{word}\t{idx}\n")
print(vocab)

In [ ]:
class SkipGram:
    def __init__(self, vocab_size, embed_size, learning_rate=0.01):
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lr = learning_rate
        
        # Khởi tạo trọng số
        self.W1 = np.random.rand(vocab_size, embed_size) 
        self.W2 = np.random.rand(embed_size, vocab_size) 

    def softmax(self, x):
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum(axis=0)

    def forward(self, center_word_idx):
        # Lớp nhúng (W1)
        h = self.W1[center_word_idx]
        # Lớp đầu ra (W2)
        u = np.dot(h, self.W2)
        y_pred = self.softmax(u)
        return y_pred, h

    def backward(self, center_word_idx, context_word_idx, y_pred, h):
        error = y_pred
        error[context_word_idx] -= 1  # Cập nhật lỗi cho từ ngữ cảnh

        # Gradient cho W2 và W1
        dW2 = np.outer(h, error)
        dW1 = np.dot(error, self.W2.T)

        # Cập nhật trọng số
        self.W2 -= self.lr * dW2
        self.W1[center_word_idx] -= self.lr * dW1


In [ ]:
def train_skipgram(model, pairs, epochs=100, batch_size=64):
    losses = []
    for epoch in range(epochs):
        total_loss = 0
        # Chia pairs thành các batch
        num_batches = len(pairs) // batch_size
        for batch_idx in range(num_batches):
            batch = pairs[batch_idx * batch_size : (batch_idx + 1) * batch_size]
            batch_loss = 0
            for center_word_idx, context_word_idx in batch:
                y_pred, h = model.forward(center_word_idx)
                #cross-entrophy
                loss = -np.log(y_pred[context_word_idx]) 
                batch_loss += loss
                model.backward(center_word_idx, context_word_idx, y_pred, h)
            total_loss += batch_loss / len(batch)  # Tính loss trung bình của batch
        losses.append(total_loss / num_batches)  # Tính loss trung bình của tất cả các batch
        print(f"Epoch {epoch+1}, Loss: {losses[-1]}")
    return losses

# Khởi tạo mô hình
embed_size = 100
skipgram_model = SkipGram(len(vocab), embed_size, learning_rate=0.01)

# Huấn luyện
losses = train_skipgram(skipgram_model, pairs, epochs=50, batch_size=128) 

# Vẽ đồ thị mất mát
import matplotlib.pyplot as plt
plt.plot(losses)
plt.title("Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()

In [ ]:
# Tính toán cosine similarity giữa hai vector
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# Cặp đồng nghĩa: ("đầu_tiên", "ban_đầu"),("bố", "cha")
# Cặp trái nghĩa: ("lên", "xuống"), ("lớn", "nhỏ")

# Kiểm tra tương đồng cosine cho các cặp từ
word_pairs = [("lên", "xuống"), ("lớn", "nhỏ"), ("đầu_tiên", "ban_đầu"),("bố", "cha")]
for word1, word2 in word_pairs:
    idx1 = vocab[word1]
    idx2 = vocab[word2]
    vec1 = skipgram_model.W1[idx1]
    vec2 = skipgram_model.W1[idx2]
    similarity = cosine_similarity(vec1, vec2)
    print(f"Cosine similarity between '{word1}' and '{word2}': {similarity}")

# Trực quan hóa vector nhúng bằng PCA cho các từ trong word_pairs
def visualize_word_pairs(skipgram_model, word_pairs, vocab, n_components=2):
    embeddings = []
    words = []

    # Lấy vector nhúng của các từ trong word_pairs
    for word1, word2 in word_pairs:
        idx1 = vocab[word1]
        idx2 = vocab[word2]
        embeddings.append(skipgram_model.W1[idx1])
        embeddings.append(skipgram_model.W1[idx2])
        words.append(word1)
        words.append(word2)

    # Áp dụng PCA để giảm chiều
    pca = PCA(n_components=n_components)
    reduced_embeddings = pca.fit_transform(np.array(embeddings))

    # Vẽ đồ thị trực quan hóa PCA
    plt.figure(figsize=(8, 8))
    for i, (x, y) in enumerate(reduced_embeddings):
        plt.scatter(x, y)
        plt.text(x + 0.01, y + 0.01, words[i], fontsize=12)

    plt.title("PCA Visualization of Word Pairs")
    plt.xlabel("Principal Component 1")
    plt.ylabel("Principal Component 2")
    plt.show()

# Gọi hàm trực quan hóa cho các cặp từ
visualize_word_pairs(skipgram_model, word_pairs, vocab)